This code takes an ERA5/CRCM6 mask and creates a csv file with the boundary layer of the CRCM6 domain. 

**Method** :  
A grid point in the mask file defines the domain's boundary if HU == True **and** there is at least one neighbouring grid point where HU == False

In [2]:
import pandas as pd
import xarray as xr

In [11]:
# Read netcdf mask file
# HU is the mask column. 
# HU == True : ERA5 grid point is within CRCM6 domain

file = '/pampa/picart/Masks/mask_GEM5_ERA5grid'
data = xr.open_dataset(file)
mask = data.to_dataframe().reset_index()

# Define the radius of neighboring grid points to consider
# ERA5 resolution : radius = 0.25
radius = 0.25

# Initialize an empty list to store the filtered grid points
filtered_points = []

# Step 1 : Iterate through each grid point in the dataframe
for i, row in mask.iterrows():
    if row['HU'] == True:
        lat = row['lat']
        lon = row['lon']

        # Step 2 : Find the neighboring grid points within the specified radius
        neighbors = mask[
            (mask['lat'] >= lat - radius) & (mask['lat'] <= lat + radius) &
            (mask['lon'] >= lon - radius) & (mask['lon'] <= lon + radius)
        ]

        # Step 3 : Check if there is at least one false 'HU' value in the neighbors
        if neighbors['HU'].isin([False]).any():
            filtered_points.append(row)

# Create a new dataframe with the filtered grid points
filtered_mask = pd.DataFrame(filtered_points)

#filtered_mask.to_csv('/pampa/cloutier/outline_crcm6_domain.csv')